# Load Train Model From Checkpoint

In [1]:
# setup paths
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CHECKPOINT_PATH = MODEL_PATH+'/detectv1/'
CUSTOM_MODEL_NAME = 'detectv1' 
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
PATH_TO_SAVED_MODEL = CHECKPOINT_PATH+'exported_models/saved_model'

In [2]:
# load dependencies
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [3]:
# load detection model
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

# Gesture Detection

In [4]:
import cv2 
import numpy as np

In [5]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [6]:
image = cv2.imread('image.jpg')
image_np = np.array(image)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.uint8)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=5,
      min_score_thresh=.5,
      agnostic_mode=False)

cv2.imshow('gesture detection',  cv2.resize(image_np_with_detections, (640, 480)))
cv2.waitKey(0)
cv2.destroyAllWindows()